In [ ]:
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode


## Install NeMo
!python -m pip install nemo_toolkit[all]

# !pip install torchtext==0.10.1
# !pip install matplotlib==3.1.3
"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""
#exit()

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! cat | ls /content/drive/MyDrive/golos/train/farfield > names.txt

^C


In [ ]:
! ls $"/content/drive/MyDrive/golos/train/farfield" > /content/drive/MyDrive/golos/train/names.txt

In [2]:
data_dir = '/content/drive/MyDrive/golos/train/farfield/'

In [3]:
import librosa
import IPython.display as ipd

In [4]:
# NeMo's "core" package
import nemo
import nemo.collections.asr as nemo_asr

[NeMo W 2021-11-29 12:53:52 optimizers:50] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [5]:
import glob
import os
import subprocess
import tarfile
import wget

def load(golos_url, fname):
  if not os.path.exists(os.path.join(data_dir, fname)):
      golos_path = wget.download(golos_url, os.path.join(data_dir, fname))
      print(f"Dataset downloaded at: {golos_path}")
  else:
      print("Tarfile already exists.")
      golos_path = os.path.join(data_dir, fname)
  return golos_path

In [6]:
load("https://sc.link/ZMv", "QuartzNet15x5_golos.nemo")
asr_model = nemo_asr.models.EncDecCTCModel.restore_from(os.path.join(data_dir, "QuartzNet15x5_golos.nemo"))

[NeMo I 2021-11-29 12:58:21 save_restore_connector:149] Model EncDecCTCModel was successfully restored from /content/drive/MyDrive/golos/train/farfield/QuartzNet15x5_golos.nemo.


In [7]:
input_names = open("/content/drive/MyDrive/golos/train/names.txt", 'r')

In [8]:
names = input_names.read()
names = names.split('\n')
names = names[:-1]

In [9]:
print(len(names))

40367


In [10]:
f2out1 = open('/content/drive/MyDrive/golos/tmp/real_tmp_names.txt', 'w')

In [11]:
true_names_map={}

In [12]:
# --- Building Manifest Files --- #
import json
real_text=[]
# Function to build a manifest
def build_manifest(manifest_rel, manifest_abs, arr=None, flag = False):
    manifest_path = os.path.split(os.path.abspath(manifest_rel))[0]
    with open(manifest_rel, 'r') as fin:
        with open(manifest_abs, 'w') as fout:
            for line in fin:
                if flag:
                  a = 8
                else:
                  a = 6
                metadata = json.loads(line)
                if metadata['id'] == '375e547e51a71f9e18b811dd89244baf':
                  continue
                metadata["audio_filepath"]=os.path.join(manifest_path, metadata["audio_filepath"][a:])
                if flag:
                  true_names_map[metadata["id"]] = metadata["text"]
                  f2out1.write(metadata["id"]+' '+metadata["text"])
                  f2out1.write('\n')
                 # pass
                 # arr.append(metadata['text'])
                json.dump(metadata, fout)
                fout.write('\n')
                if flag and metadata['id'] == '5ab4ae4de2d64d8fc0e7c12c07e37a79':
                  break

                
# Building Manifests
print("******")

build_manifest("/content/drive/MyDrive/golos/train/train_manifest.jsonl", "/content/drive/MyDrive/golos/train/outfile.jsonl", real_text, True)
train_manifest = "/content/drive/MyDrive/golos/train/outfile.jsonl"

build_manifest("/content/drive/MyDrive/golos/test/manifest.jsonl", "/content/drive/MyDrive/golos/test/outfile_test.jsonl")
test_manifest = "/content/drive/MyDrive/golos/test/outfile_test.jsonl"

******


In [ ]:
! ls "/content/drive/MyDrive/golos/test/" | grep acd44c1b8a144b89f36d357727abd6aa

In [ ]:
! cat "/content/drive/MyDrive/golos/test/manifest.jsonl" | grep acd44c1b8a144b89f36d357727abd6aa

{"id": "acd44c1b8a144b89f36d357727abd6aa", "audio_filepath": "files/acd44c1b8a144b89f36d357727abd6aa.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u043f\u0440\u0438\u043b\u043e\u0436\u0435\u043d\u0438\u0435 \u0441\u0432\u0435\u0440\u043d\u0438", "duration": 1.9996875}


In [ ]:
! head "/content/drive/MyDrive/golos/train/train_manifest.jsonl"

{"id": "77b380796d242cf5bc09cfb551cffecd", "audio_filepath": "crowd/0/77b380796d242cf5bc09cfb551cffecd.wav", "text": "\u0430\u043b\u0438\u0431\u0438", "duration": 1.21}
{"id": "404e1eecbc38a687e036739f2e5dd6d3", "audio_filepath": "crowd/0/404e1eecbc38a687e036739f2e5dd6d3.wav", "text": "\u0441\u043a\u0430\u0437\u043a\u0430", "duration": 2.9}
{"id": "f913db484595c1bcdd9476194b08a1f4", "audio_filepath": "crowd/0/f913db484595c1bcdd9476194b08a1f4.wav", "text": "\u043d\u0430\u0439\u0434\u0438 \u0432 \u043d\u0435\u0442\u0435 \u043c\u0443\u043b\u044c\u0442 \u0441\u0442\u0430\u043b\u044c\u043d\u043e\u0439 \u0433\u0438\u0433\u0430\u043d\u0442", "duration": 4.76}
{"id": "eca448acf1712fa7ca08aede3d2df1ac", "audio_filepath": "crowd/0/eca448acf1712fa7ca08aede3d2df1ac.wav", "text": "\u043c\u043e\u043d\u0430 \u043b\u0438\u0437\u0430", "duration": 2.3}
{"id": "a366eecb0a85ef76463e1f4b5188b698", "audio_filepath": "crowd/0/a366eecb0a85ef76463e1f4b5188b698.wav", "text": "\u0441\u043c\u043e\u0442\u0440\u04

In [ ]:
! cat "/content/drive/MyDrive/golos/test/outfile_test.jsonl" | grep acd44c1b8a144b89f36d357727abd6aa

{"id": "acd44c1b8a144b89f36d357727abd6aa", "audio_filepath": "/content/drive/MyDrive/golos/test/acd44c1b8a144b89f36d357727abd6aa.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u043f\u0440\u0438\u043b\u043e\u0436\u0435\u043d\u0438\u0435 \u0441\u0432\u0435\u0440\u043d\u0438", "duration": 1.9996875}


In [ ]:
! head "/content/drive/MyDrive/golos/test/outfile_test.jsonl"

{"id": "58b586f67f5e634506e215df5996b82e", "audio_filepath": "/content/drive/MyDrive/golos/test/58b586f67f5e634506e215df5996b82e.wav", "text": "\u0434\u0436\u043e\u0439 \u0445\u0432\u0430\u0442\u0438\u0442", "duration": 1.696625}
{"id": "26093ef7a8c5ec6a3c586a6a929c1bd3", "audio_filepath": "/content/drive/MyDrive/golos/test/26093ef7a8c5ec6a3c586a6a929c1bd3.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u0432\u044b\u0437\u043e\u0432 \u0441\u0432\u0435\u0442\u043b\u0430\u043d\u0435 \u0432\u0430\u0441\u0438\u043b\u044c\u0435\u0432\u043d\u0435 \u043d\u0438\u043a\u043e\u043b\u0435\u043d\u043a\u043e", "duration": 3.1720625}
{"id": "b42d13ab3dc159fac39a1120e8011f66", "audio_filepath": "/content/drive/MyDrive/golos/test/b42d13ab3dc159fac39a1120e8011f66.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u0445\u0432\u0430\u0442\u0438\u0442", "duration": 1.5226875}
{"id": "7e55843e745a239b35212e2caba77239", "audio_filepath": "/content/drive/MyDrive/golos/test/7e55843e745a239b35212e2caba77239.wav", "t

In [13]:
transcriptions2 = [] 

In [14]:
files=[]

In [ ]:
#f2out = open('/content/drive/MyDrive/golos/tmp/tmp_names.txt', 'w')

In [ ]:
#for name in transcriptions:
 # f2out.write(name)
#  f2out.write('\n')

In [15]:
input_names2 = open("/content/drive/MyDrive/golos/tmp/tmp_names.txt", 'r')

In [16]:
pred_text = input_names2.read()
pred_text = pred_text.split('\n')
pred_text = pred_text[:-1]

In [17]:
predicted_name_map = {}

In [18]:
true_names_map

{'77b380796d242cf5bc09cfb551cffecd': 'алиби',
 '404e1eecbc38a687e036739f2e5dd6d3': 'сказка',
 'f913db484595c1bcdd9476194b08a1f4': 'найди в нете мульт стальной гигант',
 'eca448acf1712fa7ca08aede3d2df1ac': 'мона лиза',
 'a366eecb0a85ef76463e1f4b5188b698': 'смотреть сериал восемь с половиной',
 '37e7ee229ce692fc007e344e6376e90a': 'вруби российский фильм мосгорсмех',
 '69c2473c78844219a6625157c0a9cb2d': 'найди на ютубе фильм меня зовут мохаммед али',
 '3d5e79e898a2d4deadda9f728fc8a3ec': 'можешь мне поставить мультфильм квадратные зверюшки',
 'da1727c120db7575744bd2740755c57d': 'сваты побыстрее включи пожалуйста',
 'ad4bbc81eb33d17f2ff22cbbd23c4eac': 'рокетмен смотреть сериал рокетмен',
 '2ecbe67fb35c14c7e8274145e3da6bfe': 'можешь поставить мировая прогулка индия гоа',
 'd954f088c73669fe21a1923a9196fee3': 'найди ка мне фильм в нете мастера секса',
 '003588da55473ff09bc6f31a9fb42aa2': 'поставь третий сезон сериала кровь за кровь',
 'a02d329c50b9abc8fdbfb54e02e90345': 'тачки три',
 'f19de139

In [19]:
len(pred_text)

39635

In [20]:
for i in range(len(pred_text)):
  if names[i][:-4] == '375e547e51a71f9e18b811dd89244baf':
    continue
  predicted_name_map[names[i][:-4]]=pred_text[i]

In [ ]:
true_names_map

In [23]:
pip install Levenshtein

     |████████████████████████████████| 110 kB 5.4 MB/s 


In [24]:
import Levenshtein as lev

In [26]:
Str1 = "Apple Inc."
Str2 = "ppl"
Distance = lev.distance(Str1.lower(),Str2.lower()),
print(Distance)
Ratio = lev.ratio(Str1.lower(),Str2.lower())
print(Ratio)

(7,)
0.4615384615384615


In [27]:
arr_for_tune = []

In [28]:
for id in names:
  try :len(predicted_name_map[id[:-4]]) 
  except KeyError: continue
  Ratio = lev.ratio(true_names_map[id[:-4]], predicted_name_map[id[:-4]])
  if Ratio < 0.75:
    arr_for_tune.append(id[:-4])

In [29]:
len(arr_for_tune)

25503

In [30]:
check_table = {}
for id in arr_for_tune:
  check_table[id] = True

In [31]:
check_table['855e01d6ba9a4aa59950d62037b87709']

True

In [32]:
# --- Building Manifest Files --- #

# Function to build a manifest
def build_manifest2(manifest_rel, manifest_abs):
    manifest_path = os.path.split(os.path.abspath(manifest_rel))[0]
    print(manifest_path)
    with open(manifest_rel, 'r') as fin:
        with open(manifest_abs, 'w') as fout:
            for line in fin:
                metadata = json.loads(line)
                if metadata['id'] in check_table:
                  metadata["audio_filepath"]=os.path.join(manifest_path, metadata["audio_filepath"])
                  json.dump(metadata, fout)
                  fout.write('\n')
                if metadata['id'] == '5ab4ae4de2d64d8fc0e7c12c07e37a79':
                  break

                
# Building Manifests
print("******")

build_manifest2("/content/drive/MyDrive/golos/train/train_manifest.jsonl", "/content/drive/MyDrive/golos/train/finetune.jsonl")
finetune_manifest = "/content/drive/MyDrive/golos/train/finetune.jsonl"



******
/content/drive/MyDrive/golos/train


In [35]:
! head /content/drive/MyDrive/golos/train/finetune.jsonl

{"id": "855e01d6ba9a4aa59950d62037b87709", "audio_filepath": "/content/drive/MyDrive/golos/train/farfield/855e01d6ba9a4aa59950d62037b87709.wav", "text": "\u0430\u0444\u0438\u043d\u0430 \u0445\u043e\u0442\u0435\u043b\u0430 \u043d\u043e\u043c\u0435\u0440 \u0442\u0435\u043b\u0435\u0444\u043e\u043d\u0430 \u043f\u043e\u043c\u0435\u043d\u044f\u0442\u044c \u043a\u0430\u0440\u0442\u044b", "duration": 5.4596875}
{"id": "5f7483035b2f26b773b0d3fb125b779d", "audio_filepath": "/content/drive/MyDrive/golos/train/farfield/5f7483035b2f26b773b0d3fb125b779d.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u044f \u043c\u043e\u0433\u0443 \u0447\u0442\u043e \u043d\u0438\u0431\u0443\u0434\u044c \u0441\u0434\u0435\u043b\u0430\u0442\u044c", "duration": 5.063625}
{"id": "4c95a12b71e1262ac937d2fe6a803906", "audio_filepath": "/content/drive/MyDrive/golos/train/farfield/4c95a12b71e1262ac937d2fe6a803906.wav", "text": "\u0434\u0436\u043e\u0439 \u0441\u043c\u043e\u0442\u0440\u0435\u0442\u044c \u043e\u043f\u0435\u0440\

In [36]:
! head /content/drive/MyDrive/golos/test/outfile_test.jsonl

{"id": "58b586f67f5e634506e215df5996b82e", "audio_filepath": "/content/drive/MyDrive/golos/test/58b586f67f5e634506e215df5996b82e.wav", "text": "\u0434\u0436\u043e\u0439 \u0445\u0432\u0430\u0442\u0438\u0442", "duration": 1.696625}
{"id": "26093ef7a8c5ec6a3c586a6a929c1bd3", "audio_filepath": "/content/drive/MyDrive/golos/test/26093ef7a8c5ec6a3c586a6a929c1bd3.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u0432\u044b\u0437\u043e\u0432 \u0441\u0432\u0435\u0442\u043b\u0430\u043d\u0435 \u0432\u0430\u0441\u0438\u043b\u044c\u0435\u0432\u043d\u0435 \u043d\u0438\u043a\u043e\u043b\u0435\u043d\u043a\u043e", "duration": 3.1720625}
{"id": "b42d13ab3dc159fac39a1120e8011f66", "audio_filepath": "/content/drive/MyDrive/golos/test/b42d13ab3dc159fac39a1120e8011f66.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u0445\u0432\u0430\u0442\u0438\u0442", "duration": 1.5226875}
{"id": "7e55843e745a239b35212e2caba77239", "audio_filepath": "/content/drive/MyDrive/golos/test/7e55843e745a239b35212e2caba77239.wav", "t

In [34]:
! cat /content/drive/MyDrive/golos/test/outfile_test.jsonl | grep acd44c1b8a144b89f36d357727abd6aa

{"id": "acd44c1b8a144b89f36d357727abd6aa", "audio_filepath": "/content/drive/MyDrive/golos/test/acd44c1b8a144b89f36d357727abd6aa.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u043f\u0440\u0438\u043b\u043e\u0436\u0435\u043d\u0438\u0435 \u0441\u0432\u0435\u0440\u043d\u0438", "duration": 1.9996875}


In [33]:
! ls /content/drive/MyDrive/golos/test | grep acd44c1b8a144b89f36d357727abd6aa

acd44c1b8a144b89f36d357727abd6aa.wav


In [37]:
params_model = asr_model._cfg

# --- Config Information ---#
# try:
#     from ruamel.yaml import YAML
# except ModuleNotFoundError:
#     from ruamel_yaml import YAML
# config_path = '/usr/local/lib/python3.7/dist-packages/nemo/collections/asr/models/configs/'

# yaml = YAML(typ='safe')
# with open(config_path) as f:
#     params = yaml.load(f)
# print(params)
params_model['train_ds']['parser']='base'
params_model['validation_ds']['parser']='base'
params_model['test_ds']['parser']='base'
params_model['train_ds']['manifest_filepath'] = finetune_manifest
params_model['train_ds']['batch_size']=16
params_model['validation_ds']['manifest_filepath'] = test_manifest
params_model['validation_ds']['batch_size']=16
params_model['test_ds']['manifest_filepath'] = test_manifest
params_model['test_ds']['batch_size']=16

In [38]:
print(test_manifest, finetune_manifest)

/content/drive/MyDrive/golos/test/outfile_test.jsonl /content/drive/MyDrive/golos/train/finetune.jsonl


In [ ]:
params_model

In [39]:
params_model['optim']['lr'] = 0.0005

In [40]:
asr_model.setup_training_data(train_data_config=params_model['train_ds'])
asr_model.setup_test_data(test_data_config=params_model['test_ds'])
# Point to the new validation data for fine-tuning
asr_model.setup_validation_data(val_data_config=params_model['validation_ds'])

[NeMo I 2021-11-29 13:01:39 collections:173] Dataset loaded with 25502 files totalling 29.44 hours
[NeMo I 2021-11-29 13:01:39 collections:174] 1 files were filtered totalling 0.00 hours


[NeMo W 2021-11-29 13:01:39 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-11-29 13:01:39 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-29 13:01:39 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-29 13:01:40 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-29 13:01:40 collections:174] 0 files were filtered totalling 0.00 hours


In [41]:
# Bigger batch-size = bigger throughput
# params['model']['validation_ds']['batch_size'] = 16
# Setup the test data loader and make sure the model is on GPU
# first_asr_model.setup_test_data(test_data_config=params['model']['validation_ds'])
asr_model.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = asr_model._wer.compute()
        asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo W 2021-11-29 13:02:04 patch_utils:50] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.
[NeMo W 2021-11-29 13:02:05 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


WER = 0.6589119541875448


In [42]:
import pytorch_lightning as pl
trainer = pl.Trainer(gpus=1, max_epochs=2, precision=16)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [43]:
trainer.fit(asr_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2021-11-29 13:10:17 modelPT:453] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-11-29 13:10:17 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0005
        weight_decay: 0.001
    )


[NeMo W 2021-11-29 13:10:17 lr_scheduler:732] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 34.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
37.859    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

[NeMo W 2021-11-29 13:10:18 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    
[NeMo W 2021-11-29 13:10:20 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [44]:
asr_model.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = asr_model._wer.compute()
        asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo W 2021-11-29 14:53:57 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


WER = 0.5565497494631353
